# Chiffrement Déterministe des Identifiants Patients

Ce script Python permet de chiffrer de manière déterministe des portions spécifiques d'identifiants patients contenus dans un fichier texte, en utilisant l'algorithme AES-256 en mode ECB avec padding PKCS7. Le chiffrement est déterministe car une même entrée produira toujours le même résultat chiffré avec la même clé.

## Sécurité de la Clé Secrète

- **Longueur minimale** : La phrase secrète doit contenir au moins 12 caractères
- **Complexité** : Utilisez un mélange de lettres (majuscules/minuscules), chiffres et caractères spéciaux
- **Confidentialité** : Cette clé doit être gardée secrète et ne pas être partagée
- **Perte de clé** : Sans la clé exacte, il est impossible de déchiffrer les données

## Caractéristiques du Chiffrement

- **Format de sortie** : Base64 URL-safe (sécurisé pour les URLs/fichiers)
- **Longueur fixe** : Toute entrée chiffrée produit **exactement 24 caractères** en sortie
- **Marqueur de fin** : Les résultats se terminent toujours par `==` (double égal)
    
    *Astuce de vérification* : Si vous avez un doute sur la longueur, comptez 24 caractères depuis la fin en incluant les `==`.

## 1. Importer les bibliothèques nécessaires

In [1]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import base64
import hashlib
import os

## 2. Fonction 1 : Génère une clé AES 256 bits à partir d'une phrase secrète

In [2]:
def generer_clef(personnalisation: str):
    cle_hash = hashlib.sha256(personnalisation.encode()).digest()
    return cle_hash[:32]  # Utiliser les 32 premiers octets pour AES-256

## 3. Fonction 2 : Chiffre l'identifiant avec padding PKCS7

In [3]:
def chiffrer_identifiant(cle, id_str):
    # Compter le nombre d'espaces à la fin
    nb_espaces_fin = len(id_str) - len(id_str.rstrip())

    # Supprimer tous les espaces
    id_str_sans_espaces = id_str.replace(" ", "")
    
    # Convertir l'identifiant en bytes
    id_bytes = id_str_sans_espaces.encode()

    # Appliquer le padding PKCS7 pour s'assurer que la longueur est multiple de 16
    padder = padding.PKCS7(128).padder()
    id_bytes_padded = padder.update(id_bytes) + padder.finalize()

    # Chiffrement AES-ECB
    cipher = Cipher(algorithms.AES(cle), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted = encryptor.update(id_bytes_padded) + encryptor.finalize()

    # Encodage base64
    encrypted_b64 = base64.urlsafe_b64encode(encrypted).decode()

    # Ajouter les espaces à la fin
    return encrypted_b64 + (" " * nb_espaces_fin)

## 4. Fonction 3 : La fonction principale

In [4]:
# Fonction principale
def main():
    print("=== Chiffrement Déterministe des Identifiants Patients ===")

    # Clé secrète
    phrase_secrete = input("Entrez votre clé secrète complexe (au moins 12 caractères, mélange de lettres/chiffres) : ")
    if len(phrase_secrete) < 12:
        print("❌ Clé trop courte. Utilisez au moins 12 caractères.")
        return
    cle = generer_clef(phrase_secrete)

    # Répertoires
    repertoire_entree = input("Entrez le répertoire contenant les fichiers à chiffrer : ").strip()
    repertoire_sortie = input("Entrez le répertoire où sauvegarder les fichiers chiffrés : ").strip()
    numero_finess = input("Entrez le numéro de FINESS : ").strip()
    annees = [2024, 2023, 2022]

    # Plages à chiffrer
    plages = [(0, 20), (40, 44)]

    fichiers_cryptes = []
    fichiers_non_trouves = []

    def chiffrer_ligne(ligne):
        ligne_mod = ligne
        offset = 0
        for debut, fin in plages:
            vrai_debut = debut + offset
            vrai_fin = fin + offset
            if vrai_fin > len(ligne_mod):
                continue
            a_chiffrer = ligne_mod[vrai_debut:vrai_fin]
            chiffre = chiffrer_identifiant(cle, a_chiffrer)
            ligne_mod = ligne_mod[:vrai_debut] + chiffre + ligne_mod[vrai_fin:]
            offset += len(chiffre) - (fin - debut)
        return ligne_mod

    for annee in annees:
        nom_fichier_entree = f"{numero_finess}.{annee}.12.tra.raa"
        chemin_entree = os.path.join(repertoire_entree, nom_fichier_entree)

        if not os.path.exists(chemin_entree):
            fichiers_non_trouves.append(nom_fichier_entree)
            continue

        try:
            data = pd.read_csv(chemin_entree, names=["colonnes"])
        except Exception as e:
            print(f"❌ Erreur de lecture {nom_fichier_entree} : {e}")
            fichiers_non_trouves.append(nom_fichier_entree)
            continue

        data["colonnes"] = data["colonnes"].apply(chiffrer_ligne)

        nom_fichier_sortie = f"TRA.RAA.{numero_finess}.{annee}.crypte.txt"
        chemin_sortie = os.path.join(repertoire_sortie, nom_fichier_sortie)

        try:
            data.to_csv(chemin_sortie, index=False, header=False)
            fichiers_cryptes.append(nom_fichier_sortie)
        except Exception as e:
            print(f"❌ Erreur lors de la sauvegarde de {nom_fichier_sortie} : {e}")
            fichiers_non_trouves.append(nom_fichier_entree)

    # Résumé final
    print("\n=== Résumé du traitement ===")
    if fichiers_cryptes:
        print("✅ Fichiers chiffrés avec succès :")
        for f in fichiers_cryptes:
            print("  -", f)
    if fichiers_non_trouves:
        print("❌ Fichiers non trouvés ou en erreur :")
        for f in fichiers_non_trouves:
            print("  -", f)

## Exécute ce bloc pour commencer

In [5]:
if __name__ == "__main__":
    main()

=== Chiffrement Déterministe des Identifiants Patients ===


Entrez votre clé secrète complexe (au moins 12 caractères, mélange de lettres/chiffres) :  xavierboucher
Entrez le répertoire contenant les fichiers à chiffrer :  C:\Users\Youba\Documents\
Entrez le répertoire où sauvegarder les fichiers chiffrés :  C:\Users\Youba\Documents\
Entrez le numéro de FINESS :  20001



=== Résumé du traitement ===
❌ Fichiers non trouvés ou en erreur :
  - 20001.2024.12.tra.raa
  - 20001.2023.12.tra.raa
  - 20001.2022.12.tra.raa


### Voir le répertoire de sauvegarde des fichiers cryptés 😊

***Préparé par : Lekehal Hammada***

***Équipe : I4S***

***20/04/2025***